# Tienhoito

### Rajapinnan tarkastelua web-sovelluksen toteuttamista varten


In [1]:
baseUrl = "https://tie.digitraffic.fi"


domains = "/api/maintenance/v1/tracking/domains" #Kunnossapidon domain
tasks = "/api/maintenance/v1/tracking/tasks" #Ajoneuvojen tehtävätyypit
routes = "/api/maintenance/v1/tracking/routes" #Ajoneuvojen seurantadata
latest = "/api/maintenance/v1/tracking/routes/latest" #Ajoneuvon viimeisin sijainti

jyvaskyla = "?xMin=25&xMax=26&yMin=61&yMax=63"

In [2]:
import requests
import pandas as pd
import json
from datetime import datetime, timedelta

#### Haetaan mahdolliset tehtävätyypit

In [3]:
response = requests.get(baseUrl+tasks)
print(baseUrl+tasks)

if response.status_code == 200:
    json_data = response.json()
    df = pd.DataFrame(json_data)
else: 
    print("Failed to fetch data")
print(df.head())

https://tie.digitraffic.fi/api/maintenance/v1/tracking/tasks
                           id                      nameFi  \
0                    BRUSHING                     Harjaus   
1              BRUSH_CLEARING  Koneellinen vesakonraivaus   
2        CLEANSING_OF_BRIDGES          Siltojen puhdistus   
3     CLEANSING_OF_REST_AREAS  L- ja p-alueiden puhdistus   
4  CLEANSING_OF_TRAFFIC_SIGNS  Liikennemerkkien puhdistus   

                       nameEn                                    nameSv  \
0                    Brushing                                 Borstning   
1              Brush clearing                              Slyklippning   
2        Cleansing of bridges                        Rengöring av broar   
3     Cleansing of rest areas  Rast- och parkeringsplatsernas rengöring   
4  Cleansing of traffic signs                 Rengöring av trafikmärken   

        dataUpdatedTime  
0  2020-03-30T00:00:00Z  
1  2020-04-09T00:00:00Z  
2  2020-04-23T00:00:00Z  
3  2020-03-30T00:

#### Haetaan domainit eli mahdolliset lähteet datalle

In [4]:
response = requests.get(baseUrl + domains)
if response.status_code == 200:
    json_data = response.json()
    df = pd.DataFrame(json_data)
else: 
    print("Failed to fetch data")
print(df)

               name                          source       dataUpdatedTime
0               all                     All domains                  None
1    municipalities        All municipality domains                  None
2       state-roads              Harja/Väylävirasto  2022-03-22T09:16:26Z
3     autori-kuopio         Autori/Kuopion kaupunki  2022-03-22T08:22:06Z
4       autori-oulu           Autori/Oulun kaupunki  2022-03-22T08:22:06Z
5  paikannin-kuopio  Paikannin.com/Kuopion kaupunki  2022-04-12T07:37:25Z


#### Jyväskylän alueella olevan aurauskaluston tämänhetkinen sijainti (Rajaus jyväskylän alueeseen koordinaatein API:n pyöristäminen huomioiden ja rajaus auraustehtäviin)

In [5]:
params = "?taskId=PLOUGHING_AND_SLUSH_REMOVAL,LEVELLING_OF_ROAD_SURFACE&xMin=25&xMax=26&yMin=61&yMax=63" 
response = requests.get(baseUrl + latest + params)
print(baseUrl + latest + params)
if response.status_code == 200:
    data = response.json()

for i in data["features"]:
    print(i["geometry"]["coordinates"][1], ",", i["geometry"]["coordinates"][0])

https://tie.digitraffic.fi/api/maintenance/v1/tracking/routes/latest?taskId=PLOUGHING_AND_SLUSH_REMOVAL,LEVELLING_OF_ROAD_SURFACE&xMin=25&xMax=26&yMin=61&yMax=63
62.349525 , 25.010638
62.236548 , 25.888741
62.565975 , 25.217393


#### Viimeisen 24 tunnin aikana suoritettujen reittien määrät tehtävätyypin mukaan
Tarkastetaan mitä tehtävänimikkeitä tyypillisesti käytetään

In [7]:
response = requests.get(baseUrl+routes+jyvaskyla)
print(baseUrl+routes+jyvaskyla)
if response.status_code == 200:
    data = response.json()

tasks = []
for i in data["features"]:
    for j in i["properties"]["tasks"]:
       tasks.append(j)
result = dict((i, tasks.count(i)) for i in tasks)
print(result)

# Reittityypit
routetypes = []
for i in data["features"]:
    routetypes.append(i["geometry"]["type"])
result2 = dict((i, routetypes.count(i)) for i in routetypes)
print(result2)

https://tie.digitraffic.fi/api/maintenance/v1/tracking/routes?xMin=25&xMax=26&yMin=61&yMax=63
{'MAINTENANCE_OF_GUIDE_SIGNS_AND_REFLECTOR_POSTS': 197, 'PLOUGHING_AND_SLUSH_REMOVAL': 3308, 'LEVELLING_OF_ROAD_SURFACE': 240, 'SALTING': 4140, 'ROAD_STATE_CHECKING': 707, 'LINE_SANDING': 17, 'CLEANSING_OF_TRAFFIC_SIGNS': 163, 'SPOT_SANDING': 67, 'LOWERING_OF_SNOWBANKS': 93}
{'Point': 1028, 'LineString': 5631}


#### Jyväskylän alueella suoritetetut aurausreitit, viimeiset 24h

In [ ]:
params = "?taskId=PLOUGHING_AND_SLUSH_REMOVAL,LEVELLING_OF_ROAD_SURFACE,LOWERING_OF_SNOWBANKS,REMOVAL_OF_BULGE_ICE&xMin=25&xMax=26&yMin=61&yMax=63"


#Aurauskaluston suorittamat reitit Jyväskylässä
response = requests.get(baseUrl + routes + params)
if response.status_code == 200:
    data = response.json()


print(baseUrl+routes+params)

print("Suoritettujen reittien määrä: ",len(data["features"]))
# Data erittäin katkonaista. Todella moni olio sisältää vain muutaman koordinaattipisteen. 
#Syyt: tehtävän muuttuminen, sijaintiviestin aikaväli yli 5 min, sijaintien välinen laskennallinen nopeus yli 140km/h

#Reitin keskimääräinen kesto ja data pisteet
durations_plough = []
num_of_data_points = []
for i in data["features"]:
    start_date = i["properties"]["startTime"]
    parsed_start_date = datetime.strptime(start_date, "%Y-%m-%dT%H:%M:%SZ")
    end_date = i["properties"]["endTime"]
    parsed_end_date = datetime.strptime(end_date, "%Y-%m-%dT%H:%M:%SZ")
    duration = parsed_end_date - parsed_start_date
    durations_plough.append(duration)
    
    #data points
    num_of_data_points.append((len(i["geometry"]["coordinates"])))


total_duration = sum(durations_plough, timedelta())
if len(durations_plough) != 0:
    average_duration = total_duration / len(durations_plough)
    print("Reitin keskimääräinen kesto: ", average_duration)
    print("Reitin datapisteiden keskiarvo: ", sum(num_of_data_points) / len(num_of_data_points))
    print ("Reitteihin käytetty aika yhteensä: ", total_duration)


https://tie.digitraffic.fi/api/maintenance/v1/tracking/routes?taskId=PLOUGHING_AND_SLUSH_REMOVAL,LEVELLING_OF_ROAD_SURFACE,LOWERING_OF_SNOWBANKS,REMOVAL_OF_BULGE_ICE&xMin=25&xMax=26&yMin=61&yMax=63
Suoritettujen reittien määrä:  3660
Reitin keskimääräinen kesto:  0:01:12.522678
Reitin datapisteiden keskiarvo:  4.184699453551913
Reitteihin käytetty aika yhteensä:  3 days, 1:43:53


#### Jyväskylän alueella suoritetut hiekoitusreitit, viimeiset 24h

In [ ]:
params = "?taskId=LINE_SANDING,SPOT_SANDING,SPREADING_OF_CRUSH&xMin=25&xMax=26&yMin=61&yMax=63"


#Aurauskaluston suorittamat reitit Jyväskylässä
response = requests.get(baseUrl + routes + params)
if response.status_code == 200:
    data = response.json()


print(baseUrl+routes+params)

print("Suoritettujen reittien määrä: ",len(data["features"]))

#Reitin keskimääräinen kesto ja data pisteet
durations_sanding = []
num_of_data_points = []
for i in data["features"]:
    start_date = i["properties"]["startTime"]
    parsed_start_date = datetime.strptime(start_date, "%Y-%m-%dT%H:%M:%SZ")
    end_date = i["properties"]["endTime"]
    parsed_end_date = datetime.strptime(end_date, "%Y-%m-%dT%H:%M:%SZ")
    duration = parsed_end_date - parsed_start_date
    durations_sanding.append(duration)
    
    #data points
    num_of_data_points.append((len(i["geometry"]["coordinates"])))


total_duration = sum(durations_sanding, timedelta())
if len(durations_sanding) != 0:
    average_duration = total_duration / len(durations_sanding)
    print("Reitin keskimääräinen kesto: ", average_duration)
    print("Reitin datapisteiden keskiarvo: ", sum(num_of_data_points) / len(num_of_data_points))
    print ("Reitteihin käytetty aika yhteensä: ", total_duration)


https://tie.digitraffic.fi/api/maintenance/v1/tracking/routes?taskId=LINE_SANDING,SPOT_SANDING,SPREADING_OF_CRUSH&xMin=25&xMax=26&yMin=61&yMax=63
Suoritettujen reittien määrä:  84
Reitin keskimääräinen kesto:  0:01:14.452381
Reitin datapisteiden keskiarvo:  6.654761904761905
Reitteihin käytetty aika yhteensä:  1:44:14


#### Jyväskylän alueella suoritetut harjausreitit, viimeiset 24h

In [ ]:
params = "?taskId=BRUSHING&xMin=25&xMax=26&yMin=61&yMax=63"


response = requests.get(baseUrl + routes + params)
if response.status_code == 200:
    data = response.json()


print(baseUrl+routes+params)

print("Suoritettujen reittien määrä: ",len(data["features"]))


#Reitin keskimääräinen kesto ja data pisteet
durations_brush = []
num_of_data_points = []
for i in data["features"]:
    start_date = i["properties"]["startTime"]
    parsed_start_date = datetime.strptime(start_date, "%Y-%m-%dT%H:%M:%SZ")
    end_date = i["properties"]["endTime"]
    parsed_end_date = datetime.strptime(end_date, "%Y-%m-%dT%H:%M:%SZ")
    duration = parsed_end_date - parsed_start_date
    durations_brush.append(duration)
    
    #data points
    num_of_data_points.append((len(i["geometry"]["coordinates"])))


total_duration = sum(durations_brush, timedelta())
if len(durations_brush) != 0:
    average_duration = total_duration / len(durations_brush)
    print("Reitin keskimääräinen kesto: ", average_duration)
    print("Reitin datapisteiden keskiarvo: ", sum(num_of_data_points) / len(num_of_data_points))
    print ("Reitteihin käytetty aika yhteensä: ", total_duration)


https://tie.digitraffic.fi/api/maintenance/v1/tracking/routes?taskId=BRUSHING&xMin=25&xMax=26&yMin=61&yMax=63
Suoritettujen reittien määrä:  0


#### Date test

In [ ]:
params = "?taskId=PLOUGHING_AND_SLUSH_REMOVAL,LEVELLING_OF_ROAD_SURFACE,LOWERING_OF_SNOWBANKS,REMOVAL_OF_BULGE_ICE&xMin=25&xMax=26&yMin=61&yMax=63"
date = "&endFrom=2024-02-15T00%3A00%3A00Z&endBefore=2024-02-15T10%3A00%3A00Z"

#Aurauskaluston suorittamat reitit Jyväskylässä
response = requests.get(baseUrl + routes + params+ date)
if response.status_code == 200:
    data = response.json()


print(baseUrl+routes+params+date)

print("Suoritettujen reittien määrä: ",len(data["features"]))
# Data erittäin katkonaista. Todella moni olio sisältää vain muutaman koordinaattipisteen. 
#Syyt: tehtävän muuttuminen, sijaintiviestin aikaväli yli 5 min, sijaintien välinen laskennallinen nopeus yli 140km/h

#Reitin keskimääräinen kesto ja data pisteet
durations_test = []
num_of_data_points = []
for i in data["features"]:
    start_date = i["properties"]["startTime"]
    parsed_start_date = datetime.strptime(start_date, "%Y-%m-%dT%H:%M:%SZ")
    end_date = i["properties"]["endTime"]
    parsed_end_date = datetime.strptime(end_date, "%Y-%m-%dT%H:%M:%SZ")
    duration = parsed_end_date - parsed_start_date
    durations_test.append(duration)
    
    #data points
    num_of_data_points.append((len(i["geometry"]["coordinates"])))


total_duration = sum(durations_test, timedelta())
if len(durations_test) != 0:
    average_duration = total_duration / len(durations_test)
    print("Reitin keskimääräinen kesto: ", average_duration)
    print("Reitin datapisteiden keskiarvo: ", sum(num_of_data_points) / len(num_of_data_points))
    print ("Reitteihin käytetty aika yhteensä: ", total_duration)

https://tie.digitraffic.fi/api/maintenance/v1/tracking/routes?taskId=PLOUGHING_AND_SLUSH_REMOVAL,LEVELLING_OF_ROAD_SURFACE,LOWERING_OF_SNOWBANKS,REMOVAL_OF_BULGE_ICE&xMin=25&xMax=26&yMin=61&yMax=63&endFrom=2024-02-15T00%3A00%3A00Z&endBefore=2024-02-15T10%3A00%3A00Z
Suoritettujen reittien määrä:  649
Reitin keskimääräinen kesto:  0:01:15.369800
Reitin datapisteiden keskiarvo:  5.064714946070878
Reitteihin käytetty aika yhteensä:  13:35:15
